# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
#Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [====================

Epoch 137/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
33/33 [==============================] - 1s 16ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 15ms/step - loss: 0.4752 - mse: 0.4752 - val_loss: 0.2357 - val_mse: 0.2357
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2480 - mse: 0.2480 - val_loss: 0.1990 - val_mse: 0.1990
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.2039 - mse: 0.2039 - val_loss: 0.1756 - val_mse: 0.1756
Epoch 4/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1718 - mse: 0.1718 - val_loss: 0.1653 - val_mse: 0.1653
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1556 - mse: 0.1556 - val_loss: 0.1671 - val_mse: 0.1671
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1418 - mse: 0.1418 - val_loss: 0.1628 - val_mse: 0.1628
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1296 - mse: 0.1296 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1222 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1686 - val_mse: 0.1686
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1713 - val_mse: 0.1713
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1710 - val_mse: 0.1710
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1743 - val_mse: 0.1743
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1707 - val_mse: 0.1707
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1701 - val_mse: 0.1701
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1704 - val_mse: 0.1704
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0228 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1759 - val_mse: 0.1759
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1774 - val_mse: 0.1774
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1784 - val_mse: 0.1784
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1787 - val_mse: 0.1787
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1788 - val_mse: 0.1788
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1794 - val_mse: 0.1794
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1797 - val_mse: 0.1797
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101


[0.010128957219421864, 0.010128957219421864]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1831 - mse: 0.1831


[0.1831451803445816, 0.1831451803445816]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

33630.18319498509

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.3465 - mse: 0.3465 - val_loss: 0.1800 - val_mse: 0.1800
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2709 - mse: 0.2709 - val_loss: 0.1988 - val_mse: 0.1988
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2195 - mse: 0.2195 - val_loss: 0.1541 - val_mse: 0.1541
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1887 - mse: 0.1887 - val_loss: 0.1837 - val_mse: 0.1837
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1709 - mse: 0.1709 - val_loss: 0.1543 - val_mse: 0.1543
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.1619 - val_mse: 0.1619
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1366 - mse: 0.1366 - val_loss: 0.1448 - val_mse: 0.1448
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1321 - 

33/33 [==============================] - 0s 8ms/step - loss: 0.0274 - mse: 0.0274 - val_loss: 0.1743 - val_mse: 0.1743
Epoch 64/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0275 - mse: 0.0275 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1614 - val_mse: 0.1614
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1554 - val_mse: 0.1554
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.1569 - val_mse: 0.1569
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1579 - val_mse: 0.1579
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0253 - mse: 0.0253 - val_loss: 0.1640 - val_mse: 0.1640
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0249 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0142 - mse: 0.0142 - val_loss: 0.1665 - val_mse: 0.1665
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0140 - mse: 0.0140 - val_loss: 0.1623 - val_mse: 0.1623
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1656 - val_mse: 0.1656
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1668 - val_mse: 0.1668
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1737 - val_mse: 0.1737
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.1615 - val_mse: 0.1615
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1661 - val_mse: 0.1661
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0116 - mse: 0.0116


[0.011603498831391335, 0.011603498831391335]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1669 - mse: 0.1669


[0.16692112386226654, 0.16692112386226654]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.4005 - mse: 0.4005 - val_loss: 0.1755 - val_mse: 0.1755
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2180 - mse: 0.2180 - val_loss: 0.1695 - val_mse: 0.1695
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1868 - mse: 0.1868 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1560 - mse: 0.1560 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1435 - mse: 0.1435 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1275 - mse: 0.1275 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1163 - mse: 0.1163 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1115 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 70/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0214 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1055 - val_mse: 0.1055
Epoch 128/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 132/150
33/33 [==============================] - 0s 6ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 2ms/step - loss: 0.0082 - mse: 0.0082


[0.008214966394007206, 0.008214966394007206]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1037 - mse: 0.1037


[0.1036883294582367, 0.1036883294582367]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.3682 - mse: 0.3682 - val_loss: 0.1491 - val_mse: 0.1491
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1665 - mse: 0.1665 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1216 - mse: 0.1216 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1022 - mse: 0.1022 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0949 - mse: 0.0949 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0631 - mse: 0.0631 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0628 - mse: 0.0628 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0461 - 

Epoch 63/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 65/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0952 - val_mse: 0.0952
Epoch 66/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.0931 - val_mse: 0.0931
Epoch 67/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 70/150
33/33 [==============================] - 0s 7ms/step - loss: 0.

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0859 - val_mse: 0.0859
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0976 - val_mse: 0.0976
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0894 - val_mse: 0.0894
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.0935 - val_mse: 0.0935
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0838 - val_mse: 0.0838
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0881 - val_mse: 0.0881
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0051 - mse: 0.0051


[0.005078520625829697, 0.005078520625829697]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0963 - mse: 0.0963


[0.09629900753498077, 0.09629900753498077]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [25]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 15ms/step - loss: 0.3544 - mse: 0.3544 - val_loss: 0.1410 - val_mse: 0.1410
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1982 - mse: 0.1982 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1297 - mse: 0.1297 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1014 - mse: 0.1014 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 5/150
33/33 [==============================] - 1s 16ms/step - loss: 0.0757 - mse: 0.0757 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 6/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0593 - mse: 0.0593 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0447 - mse: 0.0447 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0356 -

33/33 [==============================] - 0s 4ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 62/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 63/150
33/33 [==============================] - 0s 6ms/step - loss: 8.5418e-04 - mse: 8.5418e-04 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 7.6230e-04 - mse: 7.6230e-04 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 65/150
33/33 [==============================] - 0s 7ms/step - loss: 7.3840e-04 - mse: 7.3840e-04 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 8.9198e-04 - mse: 8.9198e-04 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 67/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 68/150
33/33 [==============================] - 0s 

33/33 [==============================] - 0s 4ms/step - loss: 2.7469e-04 - mse: 2.7469e-04 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 122/150
33/33 [==============================] - 0s 4ms/step - loss: 1.8943e-04 - mse: 1.8943e-04 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 123/150
33/33 [==============================] - 0s 4ms/step - loss: 2.0149e-04 - mse: 2.0149e-04 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 124/150
33/33 [==============================] - 0s 4ms/step - loss: 1.8982e-04 - mse: 1.8982e-04 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 1.8041e-04 - mse: 1.8041e-04 - val_loss: 0.1052 - val_mse: 0.1052
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 1.8818e-04 - mse: 1.8818e-04 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 1.4458e-04 - mse: 1.4458e-04 - val_loss: 0.1056 - val_mse: 0.1056
Epoch 128/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 2ms/step - loss: 0.0059 - mse: 0.0059


[0.005940326489508152, 0.005940326489508152]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0940 - mse: 0.0940


[0.09399904310703278, 0.09399904310703278]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [28]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 3ms/step - loss: 0.1389 - mse: 0.1389


[0.1388857215642929, 0.1388857215642929]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [29]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29286.016292701053

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.